# Movielens 영화 추천 실습
## 1) 데이터 준비와 전처리

In [72]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [23]:
#데이터 준비
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [24]:
#데이터 전처리
ratings = ratings[ratings['ratings']>=3] #ratings 3 미만 데이터 제거
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [25]:
#ratings 컬럼 이름 counts로 변경
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [26]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [27]:
#영화 제목을 보기 위해 메타 데이터 불러오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
#MF model을 구성하여 내가 좋아할 만한 영화를 추천

## 2) 데이터 분석

In [30]:
#ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [31]:
#ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [55]:
movie_df = pd.merge(movies, ratings)
movie_df.head()

,movie_id,title,genre,user_id,counts,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [56]:
#가장 인기있는 영화 30개(인기순)
movies_count = movie_df.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 선호하는 영화 ratings에 추가

In [57]:
movie_df[movie_df['title'].str.lower().str.contains('aladdin')].head(1)

,movie_id,title,genre,user_id,counts,timestamp
130190,588,Aladdin (1992),Animation|Children's|Comedy|Musical,1,4,978824268


In [81]:
my_favorite = ['Snow White and the Seven Dwarfs (1937)' , 'Cinderella (1950)' ,'Little Mermaid, The (1989)' ,'Beauty and the Beast (1991)' ,'Aladdin (1992)']

my_list = pd.DataFrame({'user_id': ['yena']*5, 'title': my_favorite, 'counts':[5]*5})

if not movie_df.isin({'user_id':['yena']})['user_id'].any():  
    movie_df = movie_df.append(my_list)                          

movie_df.tail(10)  

,user_id,title,counts
0,6039,574,5
1,6039,935,5
2,6039,1861,5
3,6039,575,5
4,6039,568,5
0,yena,Snow White and the Seven Dwarfs (1937),5
1,yena,Cinderella (1950),5
2,yena,"Little Mermaid, The (1989)",5
3,yena,Beauty and the Beast (1991),5
4,yena,Aladdin (1992),5


In [82]:
# 사용하는 컬럼만 남겨줍니다.
movie_df = movie_df[['user_id','title', 'counts']]
movie_df.sort_index()

,user_id,title,counts
0,0,0,5
0,yena,Snow White and the Seven Dwarfs (1937),5
0,6039,574,5
1,1,0,4
1,yena,Cinderella (1950),5
...,...,...,...
836473,2696,3627,3
836474,2161,3627,4
836475,1931,3627,3
836476,2162,3627,4


In [83]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movie_df['user_id'].unique()
movie_unique = movie_df['title'].unique()

# 유저, 아티스트 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [84]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['yena'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx['Aladdin (1992)'])

6040
3632


In [85]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movie_df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movie_df):
    print('movie column indexing OK!!')
    movie_df['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movie_df

user_id column indexing OK!!
movie column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6040,3628,5
1,6040,3629,5
2,6040,3630,5
3,6040,3631,5


## 4) CSR matrix 생성

In [86]:
num_user = movie_df['user_id'].nunique()
num_movie = movie_df['title'].nunique()

csr_data = csr_matrix((movie_df.counts, (movie_df.user_id, movie_df.title)), shape=(num_user, num_movie)) 
csr_data

<6041x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836488 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델 훈련

In [87]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [88]:
# Implicit AlternatingLeastSquares 모델의 선언, iteration은 나중에 바꿔주겠습니다.
als_model = AlternatingLeastSquares(factors=100, regularization=0.001, use_gpu=False, iterations=1000, dtype=np.float32)

In [89]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 836488 stored elements in Compressed Sparse Column format>

In [91]:
als_model.fit(csr_data_transpose) #모델 학습 

  0%|          | 0/1000 [00:00<?, ?it/s]

## 6) 훈련된 모델이 예측한 선호도를 파악하기

In [93]:
#나(yena) vector과 Aladdin vector 확인
yena, aladdin = user_to_idx['yena'], movie_to_idx['Aladdin (1992)']
yena_vector, aladdin_vector = als_model.user_factors[yena], als_model.item_factors[aladdin]

In [94]:
yena_vector

array([ 4.08206715e-12, -3.93806125e-12,  9.23544661e-12, -1.27602000e-11,
       -1.54475981e-11,  1.89627983e-11, -3.69444276e-12, -1.10539564e-11,
       -4.22686885e-12,  7.46732797e-12,  1.78872698e-11,  5.41864990e-12,
        8.61158627e-12,  8.34068838e-12,  2.16424916e-11, -1.00426065e-11,
        2.31118987e-12,  4.78030376e-12,  1.54641560e-11, -7.12546602e-12,
       -1.50061023e-11,  1.67485505e-11,  1.34367881e-11,  1.21438328e-11,
       -1.10005408e-11, -3.05985814e-12,  1.03105632e-11,  1.39987956e-12,
        1.13396055e-11, -5.91925510e-12, -1.88506658e-11,  1.32389516e-11,
        6.88869361e-12, -9.06753232e-12, -1.91037498e-11, -2.95975484e-12,
       -1.59518180e-11, -1.32520037e-11, -1.75746505e-12, -2.47771205e-12,
        1.54078954e-11, -1.51875006e-11,  1.81764499e-11, -1.09898271e-11,
       -2.01405455e-11, -1.08121428e-11, -1.25096201e-11, -2.08315179e-12,
        1.29172983e-11, -8.01960755e-12,  2.09911862e-11,  9.10443075e-12,
        1.03835985e-11,  

In [95]:
aladdin_vector

array([ 2.77602195e-15, -6.20274477e-15,  8.97280212e-15, -1.53319926e-14,
       -2.02954634e-14,  2.55040875e-14, -6.24706757e-16, -8.50959961e-15,
       -3.63897721e-15,  1.00906230e-14,  3.14061182e-14,  8.24682882e-15,
        1.98672526e-14,  2.07372283e-14,  2.96312217e-14, -9.36293872e-15,
        7.93711884e-15,  7.93803703e-15,  1.95752109e-14, -1.34764805e-14,
       -1.22079479e-14,  3.22500747e-14,  3.24715061e-14,  2.54615139e-14,
       -2.39009658e-14, -3.30318243e-15,  7.51161015e-15,  5.13048068e-15,
        1.70681798e-14, -4.97513018e-15, -2.15111623e-14,  2.05483349e-14,
        8.93855404e-15, -1.95190086e-14, -1.98430749e-14,  7.83057735e-15,
       -2.60978000e-14, -2.41135897e-14,  5.10498033e-15,  7.96148968e-15,
        1.37271566e-14, -2.64690613e-14,  2.04592660e-14, -1.33123408e-14,
       -2.22305559e-14, -1.08729173e-14, -1.27947351e-14,  2.80442635e-15,
        2.16718021e-14, -4.16987163e-15,  2.66621255e-14,  1.79052838e-14,
        1.90841436e-14,  

In [96]:
#vector을 곱했을 때의 값 확인
np.dot(yena_vector, aladdin_vector)

2.1126801e-23

In [99]:
#snow white and the seven dwarfs vector을 가지고 선호도 예측 확인
snow = movie_to_idx['Snow White and the Seven Dwarfs (1937)']
snow_vector = als_model.item_factors[snow]

print(np.dot(yena_vector, snow_vector))

1.9745824e-23


## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [110]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [112]:
get_similar_movie('Aladdin (1992)') 

['Aladdin (1992)',
 'Little Mermaid, The (1989)',
 'Beauty and the Beast (1991)',
 'Cinderella (1950)',
 'Snow White and the Seven Dwarfs (1937)',
 2930,
 2735,
 3219,
 1712,
 668]

## 8) 내가 가장 좋아할 만한 영화들을 추천받기

In [115]:
user = user_to_idx['yena']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(831, 1.1320752e-11),
 (462, 9.71701e-12),
 (832, 9.155527e-12),
 (998, 8.666855e-12),
 (1000, 8.568289e-12),
 (2053, 8.280146e-12),
 (618, 8.190935e-12),
 (343, 7.888932e-12),
 (1113, 7.879445e-12),
 (1888, 7.763419e-12),
 (2755, 7.704701e-12),
 (2480, 7.4133494e-12),
 (2724, 7.360168e-12),
 (1455, 7.092802e-12),
 (1743, 6.894456e-12),
 (2777, 6.8901967e-12),
 (2832, 6.6560663e-12),
 (823, 6.557309e-12),
 (3149, 6.515641e-12),
 (1003, 6.1880986e-12)]

In [116]:
[idx_to_movie[i[0]] for i in movie_recommended]

[831,
 462,
 832,
 998,
 1000,
 2053,
 618,
 343,
 1113,
 1888,
 2755,
 2480,
 2724,
 1455,
 1743,
 2777,
 2832,
 823,
 3149,
 1003]

In [118]:
aladdin = movie_to_idx['Aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=aladdin)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Aladdin (1992)', 2.237610434954118e-25),
 ('Little Mermaid, The (1989)', 2.17031119285293e-25),
 ('Beauty and the Beast (1991)', 2.1431556212450856e-25),
 ('Cinderella (1950)', 2.0973632291785555e-25),
 ('Snow White and the Seven Dwarfs (1937)', 2.092749492222524e-25)]